# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921682803277                   -0.69    5.8         
  2   -7.926160521798       -2.35       -1.22    1.0    230ms
  3   -7.926837304283       -3.17       -2.37    1.6    252ms
  4   -7.926861509861       -4.62       -3.04    2.9    283ms
  5   -7.926861645415       -6.87       -3.42    1.8    214ms
  6   -7.926861670456       -7.60       -3.81    1.6    211ms
  7   -7.926861680384       -8.00       -4.32    1.4    226ms
  8   -7.926861681760       -8.86       -4.82    1.8    216ms
  9   -7.926861681855      -10.02       -5.20    1.8    215ms
 10   -7.926861681871      -10.80       -5.86    1.9    219ms
 11   -7.926861681873      -11.86       -6.90    2.1    246ms
 12   -7.926861681873      -13.45       -7.40    3.0    261ms
 13   -7.926861681873   +  -15.05       -8.05    1.6    224ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921672107242                   -0.69    5.8         
  2   -7.926164812547       -2.35       -1.22    1.0    209ms
  3   -7.926837796524       -3.17       -2.37    1.6    226ms
  4   -7.926861523686       -4.62       -3.04    3.0    255ms
  5   -7.926861645534       -6.91       -3.41    1.9    245ms
  6   -7.926861669831       -7.61       -3.79    1.8    217ms
  7   -7.926861680458       -7.97       -4.34    1.2    198ms
  8   -7.926861681759       -8.89       -4.83    2.1    247ms
  9   -7.926861681864       -9.98       -5.37    1.8    213ms
 10   -7.926861681872      -11.10       -5.93    1.9    221ms
 11   -7.926861681873      -12.00       -6.86    2.0    236ms
 12   -7.926861681873      -13.45       -7.57    2.9    250ms
 13   -7.926861681873      -14.45       -8.30    2.6    237ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921672618772                   -0.69    5.8         
  2   -7.926168066587       -2.35       -1.22    1.0    205ms
  3   -7.926841145217       -3.17       -2.37    1.8    230ms
  4   -7.926864905239       -4.62       -3.04    2.5    251ms
  5   -7.926865054851       -6.83       -3.41    2.0    246ms
  6   -7.926865081464       -7.57       -3.81    1.9    212ms
  7   -7.926865091307       -8.01       -4.28    1.1    189ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
